In [1]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import csv
import ast

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
df = pd.read_csv('./SpeciesInteractions_EID2.csv')
sorted_df = df.sort_values(by='Carrier')
i = 0
current_string = ''
carrier_id = []
for idx, row in sorted_df.iterrows():
    
    if current_string != row['Carrier']:
        i+=1
        current_string = row['Carrier'] 
    carrier_id.append(i)
sorted_df['Carrier_id'] = carrier_id
sorted_df

,Cargo,Cargo classification,Carrier,Carrier classification,Sequences count,Publications count,Sequences,Publications,Carrier_id
4019,bothriocephalus acheilognathi,Helminth,abbottina rivularis,Fish,2,0,269308309;269308366,NaN,1
7729,dollfustrema hefeiensis,Helminth,abbottina rivularis,Fish,9,0,145666704;145666705;145666706;145666707;145666...,NaN,1
3494,bemisia tabaci,Arthropod,abelmoschus esculentus,Higher Plants,27,0,16565728;46016206;46016208;46016210;57790794;5...,NaN,2
2845,bacillus subtilis,Bacteria,abelmoschus esculentus,Higher Plants,1,0,294653626,NaN,2
14262,okra yellow vein mosaic virus,Virus,abelmoschus esculentus,Higher Plants,3,0,156139681;254209522;267630858,NaN,2
...,...,...,...,...,...,...,...,...,...
6113,cochliobolus geniculatus,Fungi,zoysia matrella,Higher Plants,1,0,121308950,NaN,6310
6114,cochliobolus geniculatus,Fungi,zoysia tenuifolia,Higher Plants,1,0,193811898,NaN,6311
13631,myxobolus cordeiroi,Cnidaria,zungaro jahu,Fish,1,0,238477478,NaN,6312
22213,wolbachia pipientis,Bacteria,zyginidia pullula,Arthropod,1,0,207365946,NaN,6313


In [3]:
df_grouped_carrier = sorted_df.groupby(['Carrier_id', 'Carrier', 'Carrier classification']).agg({'Cargo' : lambda x: ','.join(x)})
df_grouped_carrier['Cargo'] = df_grouped_carrier['Cargo'].str.split(',')
df_grouped_carrier.to_csv('processed_vertices.csv')
df_grouped_carrier

,,,Cargo
Carrier_id,Carrier,Carrier classification,
1,abbottina rivularis,Fish,"[bothriocephalus acheilognathi, dollfustrema h..."
2,abelmoschus esculentus,Higher Plants,"[bemisia tabaci, bacillus subtilis, okra yello..."
3,abies alba,Higher Plants,[mindarus abietinus]
4,abies balsamea,Higher Plants,[mindarus abietinus]
5,abies cephalonica,Higher Plants,[dioryctria resiniphila]
...,...,...,...
6310,zoysia matrella,Higher Plants,[cochliobolus geniculatus]
6311,zoysia tenuifolia,Higher Plants,[cochliobolus geniculatus]
6312,zungaro jahu,Fish,[myxobolus cordeiroi]


In [4]:
df_grouped_cargo = sorted_df.groupby(['Cargo']).agg({'Carrier_id' : lambda x: x.tolist()})
df_grouped_cargo.to_csv('processed_edges.csv')
df_grouped_cargo

,Carrier_id
Cargo,
'chlorella' saccharophila,[2813]
African horse sickness virus,"[1063, 2158]"
Arcanobacterium haemolyticum,[4152]
BSE agent,"[854, 2326]"
Bacteroides vulgatus,[5720]
...,...
zygocactus virus x,"[2858, 5312]"
zygops histrio,[2202]
zygops wiedii,[1124]


In [5]:
# Leitura e do arquivo e construção do grafo

shared_pathogens = Graph(directed=False) # grafo não-dirigido
shared_pathogens.add_vp('species')
shared_pathogens.add_vp('classification')
shared_pathogens.add_ep('strength')

with open('./processed_vertices.csv') as file:
    vertex_data = csv.reader(file)
    next(vertex_data, None)
    prev_i = -1
    for row in vertex_data:
        i = int(row[0])
        if i != prev_i:
            shared_pathogens.add_vertex(i-1)
        v = shared_pathogens.get_vertex(i-1)
        v['species'] = row[1]
        v['classification'] = row[2]
        prev_i = i            

In [22]:
with open('./processed_edges.csv') as file:
    edge_data = csv.reader(file)
    next(edge_data, None)
    for row in edge_data:
        connections = ast.literal_eval(row[1])
        for i in range(len(connections)):
            if i != len(connections)-1:
                next_vertex = i+1
                while(next_vertex < len(connections)):
                    if(shared_pathogens.edge_index_range == 0):
                        shared_pathogens.add_edge(i, next_vertex)
                        shared_pathogens.get_edge(i, next_vertex)['strength'] = 1
                    else if()
                    else:
                        shared_pathogens.get_edge(i, next_vertex)['strength'] += 1
                    next_vertex += 1
                    
print('número de arestas de g:', shared_pathogens.num_edges())

ValueError: Edge with source 0 and target 2 does not exist